### Homework
Ваша задача спарсить информацию о компаниях, находящихся в индексе S&P 500 с данного сайта: <br>
https://markets.businessinsider.com/index/components/s&p_500

Для каждой компании собрать следующую информацию:
* Текущая стоимость в рублях (конвертацию производить по текущему курсу, взятому с сайта [центробанка РФ](http://www.cbr.ru/development/sxml/))
* Код компании (справа от названия компании на странице компании)
* P/E компании (информация находится справа от графика на странице компании)
* Годовой рост/падение компании в процентах (основная таблица)
* Высчитать какую прибыль принесли бы акции компании (в процентах), если бы они были куплены на уровне 52 Week Low и проданы на уровне 52 Week High (справа от графика на странице компании)

Сохранить итоговую информацию в 4 JSON файла:
1. Топ 10 компаний с самими дорогими акциями в рублях.
2. Топ 10 компаний с самым низким показателем P/E.
3. Топ 10 компаний, которые показали самый высокий рост за последний год
4. Топ 10 комппаний, которые принесли бы наибольшую прибыль, если бы были куплены на самом минимуме и проданы на самом максимуме за последний год.
<br>Пример формата:
```
[
{
    "code": "MMM",
    "name": "3M CO.",
    "price" | "P/E" | "growth" | "potential profit" : value,
},
...
]
```
For scrapping you cans use `beautifulsoup4` <br>
For requesting `aiohttp`

# ВАЖНЫЙ КОММЕНТАРИЙ
Range ограничен только первыми двумя страницами (всего их 10) для большего быстродействия во время демонстрации

In [ ]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import datetime
import json

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

class Company:
    def __init__(self, name, ticker, price_rub, PE, year_stonks, year_result_percent):
        self.name = name
        self.ticker = ticker
        self.price_rub = price_rub
        self.PE = PE
        self.year_stonks = year_stonks
        self.year_result_percent = year_result_percent


companies = []

for i in range(2):
    i += 1
    url = f'https://markets.businessinsider.com/index/components/s&p_500?p={i}'
    current_date = datetime.datetime.now()
    formatted_date = current_date.strftime('%d/%m/%Y')
    cb_url = f'https://www.cbr.ru/scripts/XML_daily.asp?date_req={formatted_date}'
    cb_request = requests.get(cb_url)

    # Парсинг XML
    cb_xml = ET.fromstring(cb_request.text)

    # Поиск элемента с USD
    for valute in cb_xml.findall('Valute'):
        char_code = valute.find('CharCode').text
        if char_code == 'USD':
            value = float(valute.find('Value').text.replace(',', '.'))

    stonks_request = requests.get(url,headers=headers)

    stonks_soup = BeautifulSoup(stonks_request.text,'html.parser')

    companies_table = stonks_soup.find('tbody',class_='table__tbody')

    # Проходимся по каждой строке
    for tr in companies_table('tr'):
        
        if i == 1:
            # Получаем каждую строку
            tds = tr.find_all('td')
            
            # Достаём имя
            a_tag = tds[0].find('a') 
            name = a_tag.get('title')

            # Достаём цену в долларах
            USD_price_raw = tds[1]
            USD_price_str = USD_price_raw.get_text(strip=True, separator=' ')
            first_number = USD_price_str.split()[0]
            USD_price = float(first_number)
            # Пересчитываем в рубли
            RUB_price = round(USD_price*value,2)

            # Достаём рост/падение за год из последнего столбца
            year_result_span = tds[-1].find_all('span')
            year_stonks = year_result_span[1].get_text()

            # Достаём ссылку
            link = a_tag.get('href')

            # Переходим на страницу компании
            company_request = requests.get(f'https://markets.businessinsider.com/{link}',headers=headers)
            company_soup = BeautifulSoup(company_request.text,'html.parser')

            # Достаём ticker
            price_row = company_soup.find('h1', class_='price-section__identifiers')
            price_span = price_row.find('span', class_='price-section__category')
            mmm_span = price_span.find('span')
            ticker = mmm_span.text[2:]

            # Достаём P/E
            snapshots = company_soup.find_all('div', class_='snapshot__data-item padding-right--zero')
            # Переходим в нужный для нас snapshot 
            PE_snapshot = snapshots[14].get_text(strip = True)
            PE_metric = PE_snapshot[:-9]

            # Достаём 52 Week Low
            snapshot_small = company_soup.find('div', class_='snapshot__data-item snapshot__data-item--small')
            # Переходим в нужный для нас snapshot 
            year_week_low_snapshot = snapshot_small.get_text(strip = True)
            year_week_low = year_week_low_snapshot[:-8]

            # Достаём 52 Week High
            snapshot_big = company_soup.find('div', class_='snapshot__data-item snapshot__data-item--small snapshot__data-item--right')
            # Переходим в нужный для нас snapshot 
            year_week_high_snapshot = snapshot_big.get_text(strip = True)
            year_week_high = year_week_high_snapshot[:-9]

            # Считаем прибыль в процентах, округляя до двух знаков после запятой
            year_result_percent = round((1 - (float(year_week_low)/float(year_week_high))) * 100,2)


            # Сохраняем экземпляр класса
            company_data = [
                {'name':name,'price_rub':RUB_price,'ticker':ticker,'PE':PE_metric,'year_stonks':year_stonks,'YearResult':year_result_percent}
            ]

            companies_dict = {}

            companies.append(Company(
                name=name,
                ticker=ticker,
                price_rub=RUB_price,
                PE=PE_metric,
                year_stonks=year_stonks,
                year_result_percent=year_result_percent))



# Сортировки
# 1. Топ-10 по цене
top_price = sorted(companies, key=lambda x: x.price_rub, reverse=True)[:10]
# 2. Топ-10 по P/E
top_pe = sorted([c for c in companies if c.PE != float('inf')], key=lambda x: x.PE,reverse=True)[:10]
# 3. Топ-10 по росту
top_growth = sorted(companies, key=lambda x: x.year_stonks, reverse=True)[:10]
# 4. Топ-10 по прибыли
top_profit = sorted(companies, key=lambda x: x.year_result_percent, reverse=True)[:10]

def create_json(data, key):
    # Словарь для сопоставления ключей с атрибутами
    attr_map = {
        "price": "price_rub",
        "P/E": "PE",
        "growth": "year_stonks",
        "potential profit": "year_result_percent"
    }
    return [
        {
            "code": item.ticker,
            "name": item.name,
            key: getattr(item, attr_map[key])
        } for item in data
    ]


# Сохранение файлов
with open('JSON\\top_price.json', 'w') as f:
    json.dump(create_json(top_price, "price"), f, indent=2)

with open('JSON\\top_pe.json', 'w') as f:
    json.dump(create_json(top_pe, "P/E"), f, indent=2)

with open('JSON\\top_growth.json', 'w') as f:
    json.dump(create_json(top_growth, "growth"), f, indent=2)

with open('JSON\\top_profit.json', 'w') as f:
    json.dump(create_json(top_profit, "potential profit"), f, indent=2)